In [1]:
import pandas as pd # Used conceptually for data management, but simplified here

# --- 1. Define Hypothetical Tariff Structure ---
# NOTE: These rates are placeholders and DO NOT represent actual TGNPDCL tariffs.
# Students MUST replace these with the actual rates from their home bill.

TARIFFS = {
    "RESIDENTIAL": {  # Low Tension (LT) Customer Example
        "Fixed_Charges_Rate": 70.00,  # Per month
        "Electricity_Duty_Rate": 0.06,  # Rs. 0.06 per unit (6 paise/unit)
        "Customer_Charges": 10.00,  # Fixed per month
        "Energy_Slabs": [
            (0, 50, 3.50),    # Units 0-50: Rs 3.50/unit
            (51, 100, 4.80),  # Units 51-100: Rs 4.80/unit
            (101, 200, 6.00), # Units 101-200: Rs 6.00/unit
            (201, 99999, 7.50) # Units > 200: Rs 7.50/unit
        ]
    },
    "COMMERCIAL": {  # High Tension (HT) Customer Example
        "Fixed_Charges_Rate": 250.00, # Per month
        "Electricity_Duty_Rate": 0.10, # Rs. 0.10 per unit
        "Customer_Charges": 50.00, # Fixed per month
        "Energy_Slabs": [
            (0, 100, 7.00),    # Units 0-100: Rs 7.00/unit
            (101, 99999, 9.50) # Units > 100: Rs 9.50/unit
        ]
    }
}

# --- 2. Bill Calculation Function ---

def calculate_tgnpdcl_bill(previous_units: float, current_units: float, customer_type: str) -> dict:
    """
    Calculates the electricity bill based on consumption and customer type.
    """
    customer_type = customer_type.upper()
    if customer_type not in TARIFFS:
        return {"Error": f"Customer type '{customer_type}' is not supported in the defined tariffs."}

    # a) Read required data and calculate consumption
    PU = previous_units
    CU = current_units
    consumption = CU - PU
    
    # Check for valid consumption
    if consumption < 0:
        return {"Error": "Current units must be greater than or equal to previous units."}

    tariff = TARIFFS[customer_type]
    
    # b) Calculate bill components
    
    # i. Energy Charges (EC) - Based on Slabs
    remaining_units = consumption
    energy_charges = 0.0
    
    for start, end, rate in tariff["Energy_Slabs"]:
        if remaining_units > 0:
            units_in_slab = min(remaining_units, end - start + 1)
            # Correcting for the 0-based list logic vs the actual slab range
            if start == 0:
                # Special handling for the first slab to ensure 'up to' calculation is right
                units_in_slab = min(consumption, end)
                if consumption > end:
                    units_in_slab = end - start + 1 # Total units in slab
                else:
                    units_in_slab = consumption
            elif start > 0:
                # Standard calculation for subsequent slabs
                slab_size = end - start + 1
                units_in_slab = min(remaining_units, slab_size)

            energy_charges += units_in_slab * rate
            remaining_units -= units_in_slab

    EC = round(energy_charges, 2)

    # ii. Fixed Charges (FC)
    FC = tariff["Fixed_Charges_Rate"]

    # iii. Customer Charges (CC)
    CC = tariff["Customer_Charges"]

    # iv. Electricity Duty (ED) - Per Unit
    ED = round(consumption * tariff["Electricity_Duty_Rate"], 2)

    # v. Total Bill
    Sub_Total = EC + FC + CC + ED
    # Typically, GST/Taxes might apply here, but we'll use a simple sum
    Total_Bill = round(Sub_Total, 2)
    
    # c) Finally print the values (Return as a dict for structured output)
    
    return {
        "Consumption_Units": consumption,
        "Customer_Type": customer_type.capitalize(),
        "EC (Energy Charges)": f"Rs {EC:,.2f}",
        "FC (Fixed Charges)": f"Rs {FC:,.2f}",
        "CC (Customer Charges)": f"Rs {CC:,.2f}",
        "ED (Electricity Duty Charges)": f"Rs {ED:,.2f}",
        "Total Bill Amount": f"Rs {Total_Bill:,.2f}"
    }

# --- 3. Main Execution and Deliverable (Verification) ---

def generate_bill_output(pu: float, cu: float, type: str):
    """
    Prints the final bill output in the requested format.
    """
    print("\n" + "="*50)
    print("⚡ TGNPDCL Electricity Bill Generator (Mock-up) ⚡")
    print("="*50)
    
    result = calculate_tgnpdcl_bill(pu, cu, type)
    
    if "Error" in result:
        print(f"ERROR: {result['Error']}")
        return

    # Print input data
    print(f"**Customer Type:** {result['Customer_Type']}")
    print(f"**Previous Units (PU):** {pu:,.2f}")
    print(f"**Current Units (CU):** {cu:,.2f}")
    print(f"**Consumption Units:** {result['Consumption_Units']:,.2f} Units")
    print("-" * 50)

    # Print Calculated Charges (Expected Output Format)
    print(f"**EC (Energy Charges):** {result['EC (Energy Charges)']}")
    print(f"**FC (Fixed Charges):** {result['FC (Fixed Charges)']}")
    print(f"**CC (Customer Charges):** {result['CC (Customer Charges)']}")
    print(f"**ED (Electricity Duty Charges):** {result['ED (Electricity Duty Charges)']}")
    print("-" * 50)
    print(f"**TOTAL BILL AMOUNT DUE:** **{result['Total Bill Amount']}**")
    print("="*50)

# --- EXAMPLE USAGE (Deliverable Verification) ---
# Replace with data from your actual home bill:
# PU = Previous Units
# CU = Current Units
# Type = 'RESIDENTIAL' or 'COMMERCIAL' (based on what you configure in TARIFFS)

# Example 1: Residential Customer with 150 units consumption (50+50+50)
generate_bill_output(pu=4500, cu=4650, type='residential')

# Example 2: Commercial Customer with 300 units consumption
# generate_bill_output(pu=1000, cu=1300, type='commercial')


⚡ TGNPDCL Electricity Bill Generator (Mock-up) ⚡
**Customer Type:** Residential
**Previous Units (PU):** 4,500.00
**Current Units (CU):** 4,650.00
**Consumption Units:** 150.00 Units
--------------------------------------------------
**EC (Energy Charges):** Rs 712.50
**FC (Fixed Charges):** Rs 70.00
**CC (Customer Charges):** Rs 10.00
**ED (Electricity Duty Charges):** Rs 9.00
--------------------------------------------------
**TOTAL BILL AMOUNT DUE:** **Rs 801.50**
